# Bare Particle Composite

description

### Concepts Used
- blergh

### Tools Used
- blergh

References
------------
1. Hooper, J.B.; Schweizer, K.S.; Contact Aggregation, Bridging, and Steric Stabilization in Dense Polymer Particle Mixtures, Macromolecules 2005, 38, 8858-8869

Global Imports and Setup
---------------------------

In [ ]:
import holoviews as hv
hv.extension('bokeh')

import numpy as np

def interpolate_guess(domain_from,domain_to,rank,guess):
    '''Helper for upscaling the intial guesses'''
    guess = guess.reshape((domain_from.length,rank,rank))
    new_guess = np.zeros((domain_to.length,rank,rank))
    for i in range(rank):
        for j in range(rank):
            new_guess[:,i,j] = np.interp(domain_to.r,domain_from.r,guess[:,i,j])
    return new_guess.reshape((-1,))

In [ ]:
%opts Curve Scatter [width=500,height=400] Layout [shared_axes=False] Scatter (size=10,alpha=0.5)
%opts Curve Scatter [fontsize={'xlabel':14,'xlabel':14,'ylabel':14,'ticks':12}]
%opts Overlay [legend_position='bottom_left']
%opts Layout [shared_axes=False]


colors = {}
colors[1.0] = 'blue'
colors[0.5] = 'red'
colors[0.25] = 'green'

ls = {}
ls[1.0] = 'solid'
ls[0.5] = 'dashed'
ls[0.25] = 'dotted'

markers = {}
markers[1.0] = 'o'
markers[0.5] = '^'
markers[0.25] = 'd'

## Demo

First we load the reference data

In [ ]:
gr_compare = []
for density in [1.0,0.5,0.25]:
    fname = './data/BareComposite-Gr-alpha{}.csv'.format(density)
    x,y = np.loadtxt(fname,delimiter=',').T
    gr_compare.append([density,x,y])

Next we need to solve the PRISM equations, but we can't solve for the particle size we want (D=16d) directly. You can try it, but typyPRISM won't be able to converge the equations. Instead, we sequentially solve the PRISM equations starting for a small diameter particle and increasing. Each successive loop (or particle diameter) uses the solution from the last loop as a starting guess, greatly increasing the ability of typyPRISM to converge. 

In [ ]:
import typyPRISM
from typyPRISM.calculate.prism.pair_correlation import pair_correlation
import numpy as np

d = 1.0
phi = 0.001
eta = 0.4

sys = typyPRISM.System(['particle','polymer'],kT=1.0)
sys.domain = typyPRISM.Domain(dr=0.1,length=1024)

guess = np.zeros(sys.rank*sys.rank*sys.domain.length)
for D in np.arange(1.0,16.5,0.5):
    print('==> Solving for D=',D)
    
    vd = 4.0/3.0 * np.pi * (d/2.0)**(3)
    vD = 4.0/3.0 * np.pi * (D/2.0)**(3)
    sys.density['polymer'] = (1-phi)*eta/vd
    sys.density['particle'] = phi*eta/vD
    print('--> rho=',sys.density['polymer'],sys.density['particle'])

    sys.omega['polymer','polymer'] = typyPRISM.omega.FreelyJointedChain(N=100,l=4.0*d/3.0)
    sys.omega['polymer','particle'] = typyPRISM.omega.NoIntra()
    sys.omega['particle','particle'] = typyPRISM.omega.SingleSite()
    
    
    sys.potential['polymer','polymer'] = typyPRISM.potential.HardSphere(sigma=d)
    sys.potential['polymer','particle'] = typyPRISM.potential.Exponential(sigma=(D+d)/2.0,alpha=0.5,epsilon=1.0)
    sys.potential['particle','particle'] = typyPRISM.potential.HardSphere(sigma=D)
    
    sys.closure['polymer','polymer'] = typyPRISM.closure.PercusYevick()
    sys.closure['polymer','particle'] = typyPRISM.closure.PercusYevick()
    sys.closure['particle','particle'] = typyPRISM.closure.HyperNettedChain()

    PRISM = sys.createPRISM()

    result = PRISM.solve(guess)

    guess = np.copy(PRISM.x)

last_guess=guess


Now that we have the D=16d solution as a guess, we can use it to solve the PRISM equations for three interaction widths. 

In [ ]:
import typyPRISM
from typyPRISM.calculate.prism.pair_correlation import pair_correlation
from typyPRISM.calculate.prism.pmf import pmf
import numpy as np

sys = typyPRISM.System(['particle','polymer'],kT=1.0)
sys.domain = typyPRISM.Domain(dr=0.075,length=2048)

vd = 4.0/3.0 * np.pi * (d/2.0)**(3)
vD = 4.0/3.0 * np.pi * (D/2.0)**(3)
sys.density['polymer'] = (1-phi)*eta/vd
sys.density['particle'] = phi*eta/vD
print('--> rho=',sys.density['polymer'],sys.density['particle'])

sys.omega['polymer','polymer'] = typyPRISM.omega.FreelyJointedChain(N=100,l=4.0*d/3.0)
sys.omega['polymer','particle'] = typyPRISM.omega.NoIntra()
sys.omega['particle','particle'] = typyPRISM.omega.SingleSite()

sys.closure['polymer','polymer'] = typyPRISM.closure.PercusYevick()
sys.closure['polymer','particle'] = typyPRISM.closure.PercusYevick()
sys.closure['particle','particle'] = typyPRISM.closure.HyperNettedChain()
    
gr_results = []
guess = interpolate_guess(typyPRISM.Domain(dr=0.1,length=1024),sys.domain,sys.rank,last_guess)
for alpha in [0.25,0.5,1.0]:
    print('==> Solving for alpha=',alpha)
    sys.potential['polymer','polymer'] = typyPRISM.potential.HardSphere(sigma=d)
    sys.potential['polymer','particle'] = typyPRISM.potential.Exponential(sigma=(D+d)/2.0,alpha=alpha,epsilon=1.0)
    sys.potential['particle','particle'] = typyPRISM.potential.HardSphere(sigma=D)
    
    PRISM = sys.createPRISM()
    result = PRISM.solve(guess)
    
    x = sys.domain.r
    y = pair_correlation(PRISM)['particle','particle']
    gr_results.append([alpha,x,y])


Here we plot Figure 1 from Reference 1

In [ ]:
%%opts Overlay [legend_position='top_right']
from math import sqrt

extents = (16,0.0,24,4.0)

gr_plots = []
for alpha,x,y in gr_results:
    label = 'alpha={} (typyPRISM)'.format(alpha)
    style = {'line_dash':ls[alpha],'color':colors[alpha]}
    c1 = hv.Curve((x,y),label=label,extents=extents)(style=style)
    gr_plots.append(c1)
    
for alpha,x,y in gr_compare:
    label = 'alpha={} (Ref 1)'.format(alpha)
    style = {'marker':markers[alpha],'color':colors[alpha]}
    c1 = hv.Scatter((x,y),label=label,extents=extents)(style=style)
    gr_plots.append(c1)
    
    
hv.Overlay(gr_plots).redim.label(x='r',y='g(r)')

In [ ]:
# fig3c = {}
# for (alpha1,x1,y1),(alpha2,x2,y2) in zip(gr_results,gr_compare):
# 
#     fig3c[alpha1,'typyPRISM'] = {'x':x1,'y':y1}
#     fig3c[alpha2,'ref'] = {'x':x2,'y':y2}
# 
# import pickle
# with open('fig3c.pkl','wb') as f:
#     pickle.dump(fig3c,f)